In [ ]:
import os
import ast
import json
import time
import base64
import requests
import threading
import websocket
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timedelta, timezone
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding

load_dotenv()


# -------------------------------------------------
# CAPITAL.COM CLIENT
# -------------------------------------------------
class CapitalClient:
    CANDLES_PER_MINUTE = {
        "MINUTE": 1,
        "MINUTE_5": 1 / 5,
        "MINUTE_15": 1 / 15,
        "MINUTE_30": 1 / 30,
        "HOUR": 1 / 60,
        "HOUR_4": 1 / 240,
        "DAY": 1 / 1440,
        "WEEK": 1 / 10080,
    }

    CHUNK_MINUTES = {
        "MINUTE": 600,
        "MINUTE_5": 3000,
        "MINUTE_15": 6000,
        "MINUTE_30": 12000,
        "HOUR": 43200,
        "HOUR_4": 172800,
        "DAY": 525600,
        "WEEK": 1048320,
    }

    def __init__(self, api_key: str, identifier: str, password: str):
        self.api_key = api_key
        self.identifier = identifier
        self.password = password
        self.base_url = os.getenv("CAPITAL_BASE_URL")

        # Session management
        self.cst = None
        self.security_token = None
        self.session_expiry = None
        self._session_lock = threading.Lock()

    # ----------------------------------
    # SESSION MANAGEMENT
    # ----------------------------------
    def _is_session_valid(self) -> bool:
        """Check if the current session is still valid."""
        if not self.cst or not self.security_token:
            return False

        # If we don't have expiry time, assume it's valid
        if not self.session_expiry:
            return True

        # Add a buffer of 5 minutes before actual expiry
        return datetime.utcnow() < (self.session_expiry - timedelta(minutes=5))

    def _renew_session(self):
        """Renew the session by logging in again."""
        with self._session_lock:
            print("🔄 Renewing session...")
            self.login()

    def _ensure_valid_session(self):
        """Ensure we have a valid session, renew if needed."""
        if not self._is_session_valid():
            self._renew_session()

    # ----------------------------------
    # AUTH
    # ----------------------------------
    def login(self):
        """Login and store session expiry information."""
        r = requests.get(
            f"{self.base_url}/session/encryptionKey",
            headers={"X-CAP-API-KEY": self.api_key},
        )
        r.raise_for_status()

        encryption_key = r.json()["encryptionKey"]
        timestamp = r.json()["timeStamp"]

        message = f"{self.password}|{timestamp}".encode()
        message_b64 = base64.b64encode(message)

        public_key = serialization.load_der_public_key(base64.b64decode(encryption_key))
        encrypted = public_key.encrypt(message_b64, padding.PKCS1v15())
        encrypted_password = base64.b64encode(encrypted).decode()

        r = requests.post(
            f"{self.base_url}/session",
            headers={
                "X-CAP-API-KEY": self.api_key,
                "Content-Type": "application/json",
            },
            json={
                "identifier": self.identifier,
                "password": encrypted_password,
                "encryptedPassword": True,
            },
        )
        r.raise_for_status()

        self.cst = r.headers["CST"]
        self.security_token = r.headers["X-SECURITY-TOKEN"]

        # Try to parse expiry from response if available
        try:
            account_info = r.json().get("accountInfo", {})
            # Capital.com sessions typically last 24 hours
            self.session_expiry = datetime.utcnow() + timedelta(
                hours=23
            )  # Conservative estimate
        except:
            # If we can't parse expiry, set to 23 hours from now
            self.session_expiry = datetime.utcnow() + timedelta(hours=23)

        print("✅ Logged in successfully")

    @property
    def headers(self):
        """Get headers with automatic session renewal."""
        self._ensure_valid_session()

        if not self.cst or not self.security_token:
            raise RuntimeError("Call login() first")

        return {
            "X-CAP-API-KEY": self.api_key,
            "CST": self.cst,
            "X-SECURITY-TOKEN": self.security_token,
        }

    # ----------------------------------
    # WEB SOCKET SESSION MANAGEMENT
    # ----------------------------------
    def get_websocket_headers(self):
        """Get headers specifically for WebSocket connections."""
        self._ensure_valid_session()
        return {
            "X-CAP-API-KEY": self.api_key,
            "CST": self.cst,
            "X-SECURITY-TOKEN": self.security_token,
        }

    # ----------------------------------
    # RAW PRICE CALL (UPDATED WITH TIMEZONE HANDLING)
    # ----------------------------------
    def _fetch_prices(
        self,
        epic: str,
        resolution: str,
        start: datetime,
        end: datetime,
    ) -> pd.DataFrame:

        minutes = (end - start).total_seconds() / 60
        max_points = int(minutes * self.CANDLES_PER_MINUTE[resolution]) + 5

        # Convert to UTC and r emove timezone info for Capital.com API
        if start.tzinfo is not None:
            start = start.astimezone(timezone.utc).replace(tzinfo=None)

        if end.tzinfo is not None:
            end = end.astimezone(timezone.utc).replace(tzinfo=None)

        # Format as simple ISO string without timezone
        start_iso = start.strftime("%Y-%m-%dT%H:%M:%S")
        end_iso = end.strftime("%Y-%m-%dT%H:%M:%S")

        params = {
            "resolution": resolution,
            "from": start_iso,
            "to": end_iso,
            "max": max_points,
        }

        r = requests.get(
            f"{self.base_url}/prices/{epic}",
            headers=self.headers,
            params=params,
        )
        r.raise_for_status()

        prices = r.json().get("prices", [])
        if not prices:
            return pd.DataFrame()

        df = pd.DataFrame(prices)

        # Convert to UTC timezone-aware datetime
        df["timestamp"] = pd.to_datetime(df["snapshotTime"]).dt.tz_localize(None)

        # For 15-minute candles, check if we need to round to the nearest 15 minutes
        if resolution == "MINUTE_15":
            # Round down to the nearest 15 minutes
            df["timestamp"] = df["timestamp"].dt.floor("15min")

        df = df.sort_values("timestamp")

        return df[["timestamp", "openPrice", "highPrice", "lowPrice", "closePrice"]]

    # ----------------------------------
    # SMART HISTORICAL DOWNLOAD (UPDATED)
    # ----------------------------------
    def get_historical_prices(
        self,
        epic: str,
        resolution: str,
        from_date: str,
        to_date: str,
        timezone_offset: int = 0,  # Timezone offset in hours (e.g., 5.5 for IST)
    ) -> pd.DataFrame:
        """
        Get historical prices with timezone handling.
        """

        # Parse dates (handle both date-only and datetime strings)
        try:
            start_dt = datetime.fromisoformat(from_date)
        except ValueError:
            # If only date is provided, add time
            start_dt = datetime.strptime(from_date, "%Y-%m-%d")

        try:
            end_dt = datetime.fromisoformat(to_date)
        except ValueError:
            # If only date is provided, add end of day
            end_dt = datetime.strptime(to_date, "%Y-%m-%d")
            end_dt = end_dt.replace(hour=23, minute=59, second=59, microsecond=999999)

        # Apply timezone offset if needed
        if timezone_offset != 0:
            start_dt = start_dt + timedelta(hours=timezone_offset)
            end_dt = end_dt + timedelta(hours=timezone_offset)
            print(f"📅 Applied {timezone_offset:+} hour timezone offset")

        delta = timedelta(minutes=self.CHUNK_MINUTES[resolution])

        all_chunks = []
        current = start_dt

        print(f"\n📥 Downloading {epic} {resolution} candles")
        print(
            f"   Date range: {start_dt.strftime('%Y-%m-%d %H:%M:%S')} to {end_dt.strftime('%Y-%m-%d %H:%M:%S')}"
        )

        while current < end_dt:
            chunk_end = min(current + delta, end_dt)

            print(
                f"  {current.strftime('%Y-%m-%d %H:%M:%S')} → {chunk_end.strftime('%Y-%m-%d %H:%M:%S')}"
            )

            df = self._fetch_prices(
                epic=epic,
                resolution=resolution,
                start=current,
                end=chunk_end,
            )

            if not df.empty:
                all_chunks.append(df)

            current = chunk_end

        if not all_chunks:
            return pd.DataFrame()

        df = (
            pd.concat(all_chunks)
            .drop_duplicates("timestamp")
            .sort_values("timestamp")
            .reset_index(drop=True)
        )

        # Apply reverse timezone offset to display in local time
        if timezone_offset != 0:
            df["timestamp"] = df["timestamp"] - pd.Timedelta(hours=timezone_offset)

        print(
            f"✅ Done: {len(df):,} candles "
            f"({df.timestamp.min().strftime('%Y-%m-%d %H:%M:%S')} → {df.timestamp.max().strftime('%Y-%m-%d %H:%M:%S')})"
        )

        # Check for missing candles
        if len(df) > 1:
            expected_freq = "15min" if resolution == "MINUTE_15" else "1D"
            full_range = pd.date_range(
                start=df.timestamp.min(), end=df.timestamp.max(), freq=expected_freq
            )
            missing = set(full_range) - set(df.timestamp)
            if missing:
                print(f"⚠️  Missing {len(missing)} candles at: {list(missing)[:5]}...")

        return df

    # ----------------------------------
    # ACCOUNTS (BALANCE / MARGIN / P&L)
    # ----------------------------------
    def get_accounts(self) -> pd.DataFrame:
        """
        Fetch all trading accounts with balances and margin info.
        """
        print(self.base_url)
        r = requests.get(
            f"{self.base_url}/accounts",
            headers=self.headers,
        )
        r.raise_for_status()

        accounts = r.json().get("accounts", [])
        if not accounts:
            return pd.DataFrame()

        df = pd.json_normalize(accounts)

        # Optional: cleaner column names
        df = df.rename(
            columns={
                "balance.balance": "balance",
                "balance.available": "available",
                "balance.deposit": "deposit",
                "balance.profitLoss": "profit_loss",
            }
        )

        return df

    # ----------------------------------
    # MARKET SEARCH
    # ----------------------------------
    def search_markets(self, search_term=""):
        r = requests.get(
            f"{self.base_url}/markets",
            headers=self.headers,
            params={"searchTerm": search_term, "limit": 20},
        )
        r.raise_for_status()
        return r.json().get("markets", [])

        # ----------------------------------

    # WORKING ORDERS (LIMIT/STOP ORDERS)
    # ----------------------------------
    def create_working_order(
        self,
        epic: str,
        direction: str,
        size: float,
        level: float,
        order_type: str,
        good_till_date: str = None,
        guaranteed_stop: bool = False,
        trailing_stop: bool = False,
        stop_level: float = None,
        stop_distance: float = None,
        stop_amount: float = None,
        profit_level: float = None,
        profit_distance: float = None,
        profit_amount: float = None,
        deal_reference: str = None,
    ) -> dict:
        """Create a limit or stop order (working order)."""

        # Validate required parameters
        if direction not in ["BUY", "SELL"]:
            raise ValueError("direction must be either 'BUY' or 'SELL'")

        if size <= 0:
            raise ValueError("size must be greater than 0")

        if level <= 0:
            raise ValueError("level must be greater than 0")

        if order_type not in ["LIMIT", "STOP"]:
            raise ValueError("order_type must be either 'LIMIT' or 'STOP'")

        # Validate stop parameters
        if guaranteed_stop and trailing_stop:
            raise ValueError("Cannot set both guaranteedStop and trailingStop to True")

        if guaranteed_stop:
            if not any([stop_level, stop_distance, stop_amount]):
                raise ValueError(
                    "If guaranteedStop=True, must provide stopLevel, stopDistance, or stopAmount"
                )

        if trailing_stop:
            if stop_distance is None:
                raise ValueError("If trailingStop=True, must provide stopDistance")

        # Validate good_till_date format if provided
        if good_till_date:
            try:
                datetime.fromisoformat(good_till_date.replace("Z", ""))
            except ValueError:
                raise ValueError("good_till_date must be in format YYYY-MM-DDTHH:MM:SS")

        # Prepare request body
        body = {
            "epic": epic,
            "direction": direction,
            "size": size,
            "level": level,
            "type": order_type,
            "guaranteedStop": guaranteed_stop,
            "trailingStop": trailing_stop,
        }

        # Add optional parameters if provided
        if good_till_date is not None:
            body["goodTillDate"] = good_till_date

        if stop_level is not None:
            body["stopLevel"] = stop_level

        if stop_distance is not None:
            body["stopDistance"] = stop_distance

        if stop_amount is not None:
            body["stopAmount"] = stop_amount

        if profit_level is not None:
            body["profitLevel"] = profit_level

        if profit_distance is not None:
            body["profitDistance"] = profit_distance

        if profit_amount is not None:
            body["profitAmount"] = profit_amount

        if deal_reference is not None:
            body["dealReference"] = deal_reference

        # Make the API request
        r = requests.post(
            f"{self.base_url}/workingorders", headers=self.headers, json=body
        )
        r.raise_for_status()

        response = r.json()

        # Add convenience fields
        if "dealReference" in response:
            response["order_id"] = response["dealReference"].replace("o_", "")
            response["dealId"] = response["order_id"]

        order_type_desc = "Limit" if order_type == "LIMIT" else "Stop"
        print(
            f"✅ {order_type_desc} order created: {direction} {size} {epic} @ {level}"
        )
        if "dealReference" in response:
            print(f"   Deal Reference: {response['dealReference']}")

        return response

    # ----------------------------------
    # GET ALL WORKING ORDERS
    # ----------------------------------
    def get_working_orders(self) -> pd.DataFrame:
        """
        Get all pending working orders (limit/stop orders).

        """
        r = requests.get(f"{self.base_url}/workingorders", headers=self.headers)
        r.raise_for_status()

        orders = r.json().get("workingOrders", [])
        if not orders:
            return pd.DataFrame()

        df = pd.json_normalize(orders)

        # Optional: Clean up column names
        df = df.rename(columns=lambda x: x.replace(".", "_"))

        # Convert timestamp columns if they exist
        timestamp_cols = ["createdDate", "goodTillDate"]
        for col in timestamp_cols:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col])

        return df

    # ----------------------------------
    # GET SPECIFIC WORKING ORDER
    # ----------------------------------
    def get_working_order(self, deal_id: str) -> dict:
        """
        Get details of a specific working order.
        """
        r = requests.get(
            f"{self.base_url}/workingorders/{deal_id}", headers=self.headers
        )
        r.raise_for_status()
        return r.json()

    # ----------------------------------
    # UPDATE WORKING ORDER
    # ----------------------------------
    def update_working_order(
        self,
        deal_id: str,
        level: float = None,
        good_till_date: str = None,
        guaranteed_stop: bool = None,
        trailing_stop: bool = None,
        stop_level: float = None,
        stop_distance: float = None,
        stop_amount: float = None,
        profit_level: float = None,
        profit_distance: float = None,
        profit_amount: float = None,
    ) -> dict:
        """
        Update a limit or stop working order.

        """

        # Validate parameter combinations
        if guaranteed_stop is not None and trailing_stop is not None:
            if guaranteed_stop and trailing_stop:
                raise ValueError(
                    "Cannot set both guaranteedStop and trailingStop to True"
                )

        if guaranteed_stop and guaranteed_stop is True:
            if not any([stop_level, stop_distance, stop_amount]):
                raise ValueError(
                    "If guaranteedStop=True, must provide stopLevel, stopDistance, or stopAmount"
                )

        if trailing_stop and trailing_stop is True:
            if stop_distance is None:
                raise ValueError("If trailingStop=True, must provide stopDistance")

        # Validate good_till_date format if provided
        if good_till_date:
            try:
                datetime.fromisoformat(good_till_date.replace("Z", ""))
            except ValueError:
                raise ValueError("good_till_date must be in format YYYY-MM-DDTHH:MM:SS")

        # Prepare request body
        body = {}

        # Add parameters if provided
        if level is not None:
            if level <= 0:
                raise ValueError("level must be greater than 0")
            body["level"] = level

        if good_till_date is not None:
            body["goodTillDate"] = good_till_date

        if guaranteed_stop is not None:
            body["guaranteedStop"] = guaranteed_stop

        if trailing_stop is not None:
            body["trailingStop"] = trailing_stop

        if stop_level is not None:
            body["stopLevel"] = stop_level

        if stop_distance is not None:
            body["stopDistance"] = stop_distance

        if stop_amount is not None:
            body["stopAmount"] = stop_amount

        if profit_level is not None:
            body["profitLevel"] = profit_level

        if profit_distance is not None:
            body["profitDistance"] = profit_distance

        if profit_amount is not None:
            body["profitAmount"] = profit_amount

        # Make the API request
        r = requests.put(
            f"{self.base_url}/workingorders/{deal_id}", headers=self.headers, json=body
        )
        r.raise_for_status()

        response = r.json()
        print(f"✅ Working order updated: Deal ID {deal_id}")

        return response

    # ----------------------------------
    # DELETE WORKING ORDER
    # ----------------------------------
    def delete_working_order(self, deal_id: str) -> dict:
        """
        Delete (cancel) a working order.
        """
        r = requests.delete(
            f"{self.base_url}/workingorders/{deal_id}", headers=self.headers
        )
        r.raise_for_status()

        response = r.json()
        print(f"✅ Working order deleted: Deal ID {deal_id}")

        return response

    # ----------------------------------
    # WEBSOCKET OHLC STREAM (UPDATED)
    # ----------------------------------
    def stream_ohlc(
        self,
        epics: list[str],
        resolution: str = "MINUTE_15",
        candle_type: str = "classic",
        on_candle_close=None,
        auto_reconnect: bool = True,
        reconnect_delay: int = 5,
    ):
        """Enhanced WebSocket streaming with session management."""
        if not on_candle_close:
            raise ValueError("on_candle_close callback is required")

        ws_url = "wss://api-streaming-capital.backend-capital.com/connect"

        # State management
        self._ws_active = False
        self._ws_stop = threading.Event()

        # Candle tracking
        last_ts = {}
        last_candle = {}

        def on_message(ws, raw):
            try:
                msg = json.loads(raw)

                # Handle ping response
                if msg.get("destination") == "ping":
                    # print("🏓 Ping received")
                    return

                # Handle subscription response
                if msg.get("destination") == "OHLCMarketData.subscribe":
                    status = msg.get("status")
                    if status == "OK":
                        subscriptions = msg.get("payload", {}).get("subscriptions", {})
                        print(f"✅ Subscription successful: {subscriptions}")
                    else:
                        error_code = msg.get("payload", {}).get("errorCode")
                        print(f"❌ Subscription failed: {error_code}")

                        # Handle specific errors
                        if error_code == "error.invalid.session.token":
                            print("🔄 Session expired, renewing...")
                            self._renew_session()
                            # Resubscribe with new tokens
                            subscribe_to_ohlc(ws)
                    return

                # Handle OHLC data
                if msg.get("destination") == "ohlc.event":
                    p = msg["payload"]
                    epic = p["epic"]
                    ts = p["t"]
                    res = p.get("resolution", "UNKNOWN")

                    # Debug print (comment out in production)
                    # print(f"📊 {epic} {res} update: {p['c']}")

                    # Check if this is a new candle (different timestamp)
                    if epic in last_ts and ts != last_ts[epic]:
                        # Previous candle closed
                        if epic in last_candle:
                            closed_candle = last_candle[epic].copy()
                            closed_candle["timestamp"] = datetime.fromtimestamp(
                                closed_candle["t"] / 1000, tz=datetime.utcnow().tzinfo
                            )
                            closed_candle["epic"] = epic
                            closed_candle["resolution"] = last_candle[epic].get(
                                "resolution", res
                            )
                            on_candle_close(closed_candle)

                    # Update last values
                    last_ts[epic] = ts
                    last_candle[epic] = p

                # Handle general subscription responses
                elif msg.get("destination") == "marketData.subscribe":
                    status = msg.get("status")
                    if status == "OK":
                        pass  # Just acknowledge

            except Exception as e:
                print(f"❌ Error processing message: {e}")
                import traceback

                traceback.print_exc()

        def subscribe_to_ohlc(ws):
            """Helper function to subscribe to OHLC data."""
            ws_headers = self.get_websocket_headers()

            sub_msg = {
                "destination": "OHLCMarketData.subscribe",
                "correlationId": str(int(time.time() * 1000)),
                "cst": ws_headers["CST"],
                "securityToken": ws_headers["X-SECURITY-TOKEN"],
                "payload": {
                    "epics": epics,
                    "resolutions": [resolution],
                    "type": candle_type,
                },
            }
            ws.send(json.dumps(sub_msg))
            print(f"📡 Sent subscription for {epics} {resolution}")

        def on_open(ws):
            print("✅ WebSocket connection opened")

            # Send ping with current session tokens
            ws_headers = self.get_websocket_headers()

            ping_msg = {
                "destination": "ping",
                "correlationId": str(int(time.time() * 1000)),
                "cst": ws_headers["CST"],
                "securityToken": ws_headers["X-SECURITY-TOKEN"],
            }
            ws.send(json.dumps(ping_msg))

            # Wait a moment
            time.sleep(0.5)

            # Subscribe to OHLC data
            subscribe_to_ohlc(ws)

            self._ws_active = True

        def on_error(ws, error):
            print(f"❌ WebSocket error: {error}")
            if auto_reconnect and not self._ws_stop.is_set():
                print(f"⏳ Reconnecting in {reconnect_delay} seconds...")
                time.sleep(reconnect_delay)
                connect_websocket()

        def on_close(ws, close_status_code, close_msg):
            print(f"🔌 WebSocket closed: Code={close_status_code}")
            self._ws_active = False

            if auto_reconnect and not self._ws_stop.is_set():
                print(f"⏳ Reconnecting in {reconnect_delay} seconds...")
                time.sleep(reconnect_delay)
                connect_websocket()

        def connect_websocket():
            """Connect to WebSocket with current session."""
            print(f"🌐 Connecting to WebSocket...")

            # Get fresh session headers
            ws_headers = self.get_websocket_headers()

            ws = websocket.WebSocketApp(
                ws_url,
                on_open=on_open,
                on_message=on_message,
                on_error=on_error,
                on_close=on_close,
                header=[
                    f"X-CAP-API-KEY: {self.api_key}",
                    f"CST: {ws_headers['CST']}",
                    f"X-SECURITY-TOKEN: {ws_headers['X-SECURITY-TOKEN']}",
                ],
            )

            # Run in background thread
            def run_ws():
                ws.run_forever(
                    ping_interval=30,  # Send ping every 30 seconds
                    ping_timeout=10,
                    ping_payload=json.dumps(
                        {
                            "destination": "ping",
                            "correlationId": str(int(time.time() * 1000)),
                        }
                    ),
                    reconnect=5,
                )

            thread = threading.Thread(target=run_ws, daemon=True)
            thread.start()
            return ws

        # Start the WebSocket connection
        return connect_websocket()

    def stop_streaming(self):
        """Stop the WebSocket streaming."""
        self._ws_stop.set()
        self._ws_active = False

In [ ]:
if __name__ == "__main__":
    client = CapitalClient(
        api_key=os.getenv("CAPITAL_DEMO_API_KEY"),
        identifier=os.getenv("CAPITAL_IDENTIFIER"),
        password=os.getenv("CAPITAL_PASSWORD"),
    )

    client.login()

    # Search for gold
    markets = client.search_markets("gold")

    if markets:
        epic_to_use = markets[0]["epic"]
        print(f"\n✅ Using epic: {epic_to_use}")

        # Get market info to check current price and requirements
        market_info = markets[0]
        print(f"\nMarket Info:")
        print(f"  Epic: {market_info.get('epic')}")
        print(f"  Instrument Name: {market_info.get('instrumentName')}")
        print(f"  Min Deal Size: {market_info.get('minDealSize')}")
        print(f"  Deal Size Increment: {market_info.get('dealSizeIncrement')}")

    try:

        response = client.create_working_order(
            epic=epic_to_use,
            direction="SELL",
            size=0.01,
            level=5159.00,  # Entry  price
            order_type="STOP",
            stop_level=5274,  # SL above entry (stop if price goes UP)
            profit_level=5100,  # TP below entry (profit if price goes DOWN)
            trailing_stop=False,
            guaranteed_stop=False,
        )
        print(f"\n✅ Order created successfully:")
        print(json.dumps(response, indent=2))

    except requests.exceptions.HTTPError as e:
        print(f"\n❌ Error: {e}")
        print(f"Status: {e.response.status_code}")
        print(f"Response: {e.response.text}")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11984\982383354.py:123: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.session_expiry = datetime.utcnow() + timedelta(
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11984\982383354.py:68: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow() < (self.session_expiry - timedelta(minutes=5))


✅ Logged in successfully

✅ Using epic: GOLD

Market Info:
  Epic: GOLD
  Instrument Name: Gold
  Min Deal Size: None
  Deal Size Increment: None
✅ Stop order created: SELL 0.01 GOLD @ 5159.0
   Deal Reference: o_200b75e0-8bf5-4775-8ac8-19f823a79697

✅ Order created successfully:
{
  "dealReference": "o_200b75e0-8bf5-4775-8ac8-19f823a79697",
  "order_id": "200b75e0-8bf5-4775-8ac8-19f823a79697",
  "dealId": "200b75e0-8bf5-4775-8ac8-19f823a79697"
}


In [ ]:
import os
import json
import requests
from dotenv import load_dotenv

if __name__ == "__main__":
    load_dotenv()

    client = CapitalClient(
        api_key=os.getenv("CAPITAL_DEMO_API_KEY"),
        identifier=os.getenv("CAPITAL_IDENTIFIER"),
        password=os.getenv("CAPITAL_PASSWORD"),
    )

    client.login()

    # Search for gold
    markets = client.search_markets("gold")

    if not markets:
        print("❌ No gold markets found")
        exit()

    epic_to_use = markets[0]["epic"]
    market_info = markets[0]

    print(f"\n✅ Using epic: {epic_to_use}")
    print(f"\nMarket Info:")
    print(f"  Instrument: {market_info.get('instrumentName')}")
    print(f"  Bid: {market_info.get('bid')}")
    print(f"  Ask: {market_info.get('ask')}")
    print(f"  Spread: {market_info.get('offer') - market_info.get('bid'):.2f}")
    print(f"  Min Deal Size: {market_info.get('minDealSize')}")

    # Get current market price for reference
    current_bid = market_info.get("bid")
    current_ask = market_info.get("ask")
    current_price = (current_bid + current_ask) / 2

    print(f"\n📊 Current Market Price:")
    print(f"  Bid: {current_bid}")
    print(f"  Ask: {current_ask}")
    print(f"  Mid: {current_price:.2f}")

    # For a SELL STOP order:
    # - Entry (level) must be BELOW current price
    # - Stop Loss (stop_level) must be ABOVE entry
    # - Take Profit (profit_level) must be BELOW entry

    # Calculate appropriate levels
    entry_price = 5250  # Your desired entry

    # IMPORTANT: For STOP orders, check if this makes sense
    if entry_price > current_price:
        print(
            f"\n⚠️  WARNING: For a SELL STOP order, entry should be BELOW current price"
        )
        print(f"   Current price: {current_price}")
        print(f"   Your entry: {entry_price}")
        print(
            f"   Consider using LIMIT order instead if you want to sell above current price"
        )

    # Calculate stops with proper distance
    # Stop Loss should be above entry (for sell order)
    stop_loss = entry_price + 20  # 20 points above entry

    # Take Profit should be below entry
    take_profit = entry_price - 50  # 50 points below entry

    print(f"\n📝 Order Parameters:")
    print(f"  Entry Price (level): {entry_price}")
    print(f"  Stop Loss: {stop_loss} (+{stop_loss - entry_price} points)")
    print(f"  Take Profit: {take_profit} (-{entry_price - take_profit} points)")

    # Check if using stop_distance instead of stop_level might work better
    print(
        "\nℹ️  Note: Capital.com might require stopDistance instead of stopLevel for certain instruments"
    )

    try:
        # Try with stop_distance instead of stop_level
        response = client.create_working_order(
            epic=epic_to_use,
            direction="SELL",
            size=0.01,
            level=entry_price,
            order_type="STOP",
            # Try using stop_distance instead of stop_level
            stop_distance=20,  # 20 points from entry
            profit_level=take_profit,
            trailing_stop=False,
            guaranteed_stop=False,
        )
        print(f"\n✅ Order created successfully!")
        print(json.dumps(response, indent=2))

    except requests.exceptions.HTTPError as e:
        print(f"\n❌ Error: {e}")
        print(f"Status: {e.response.status_code}")
        response_text = e.response.text

        try:
            error_json = json.loads(response_text)
            print(f"Error details: {json.dumps(error_json, indent=2)}")
        except:
            print(f"Response: {response_text}")

        print("\n🔄 Trying alternative approach...")

        # Alternative 1: Try with larger stop distance
        try:
            response = client.create_working_order(
                epic=epic_to_use,
                direction="SELL",
                size=0.01,
                level=entry_price,
                order_type="STOP",
                stop_distance=50,  # Larger distance
                profit_level=take_profit,
                trailing_stop=False,
                guaranteed_stop=False,
            )
            print(f"\n✅ Order created with larger stop distance!")
            print(json.dumps(response, indent=2))
        except Exception as e2:
            print(f"❌ Alternative 1 failed: {e2}")

            # Alternative 2: Try creating a LIMIT order instead
            try:
                print("\n🔄 Trying LIMIT order instead...")
                response = client.create_working_order(
                    epic=epic_to_use,
                    direction="SELL",
                    size=0.01,
                    level=entry_price,
                    order_type="LIMIT",  # Change to LIMIT
                    stop_distance=20,
                    profit_level=take_profit,
                    trailing_stop=False,
                    guaranteed_stop=False,
                )
                print(f"\n✅ LIMIT order created successfully!")
                print(json.dumps(response, indent=2))
            except Exception as e3:
                print(f"❌ Alternative 2 failed: {e3}")

                # Alternative 3: Try creating position instead of working order
                try:
                    print("\n🔄 Creating immediate position instead...")
                    response = client.create_position(
                        epic=epic_to_use,
                        direction="SELL",
                        size=0.01,
                        stop_level=stop_loss,
                        profit_level=take_profit,
                        guaranteed_stop=False,
                        trailing_stop=False,
                    )
                    print(f"\n✅ Position created successfully!")
                    print(json.dumps(response, indent=2))
                except Exception as e4:
                    print(f"❌ Alternative 3 failed: {e4}")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11984\982383354.py:123: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.session_expiry = datetime.utcnow() + timedelta(
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11984\982383354.py:68: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow() < (self.session_expiry - timedelta(minutes=5))


✅ Logged in successfully

✅ Using epic: GOLD

Market Info:
  Instrument: Gold
  Bid: 5184.76
  Ask: None
  Spread: 0.75
  Min Deal Size: None


TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

In [ ]:
 
 if __name__ == "__main__":
    # Initialize client
    client = CapitalClient(
        api_key=os.getenv("CAPITAL_DEMO_API_KEY"),
        identifier=os.getenv("CAPITAL_IDENTIFIER"),
        password=os.getenv("CAPITAL_PASSWORD"),
    )

    # Login initially
    client.login()

    def on_candle_closed(candle):
        print("\n" + "=" * 60)
        print(f"🕯️ CLOSED CANDLE: {candle['epic']}")
        print(f"📅 Time: {candle['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}")

        # Use single-letter keys from Capital.com's payload
        print(
            f"📊 O:{candle['o']:.5f} H:{candle['h']:.5f} L:{candle['l']:.5f} C:{candle['c']:.5f}"
        )
        print(f"🔧 Resolution: {candle.get('resolution', 'N/A')}")
        print("=" * 60)

    # Search for markets
    try:
        markets = client.search_markets("gold")

        if markets:
            epic_to_use = markets[0]["epic"]
            print(f"\n✅ Using epic: {epic_to_use}")
        else:
            epic_to_use = "CS.D.EURUSD.MINI.IP"  # Fallback
            print(f"\n⚠️  No markets found, using: {epic_to_use}")

    except Exception as e:
        print(f"❌ Could not fetch markets: {e}")
        epic_to_use = "CS.D.EURUSD.MINI.IP"  # Default fallback

    # Start WebSocket
    print(f"\n🚀 Starting WebSocket for: {epic_to_use}")
    ws = client.stream_ohlc(
        epics=[epic_to_use],
        resolution="MINUTE",
        on_candle_close=on_candle_closed,
        auto_reconnect=True,
        reconnect_delay=5,
    )

    # Keep main thread alive
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\n👋 Stopping WebSocket...")
        client.stop_streaming()

✅ Found gold_yesterday_levels.csv file
⚠️  CapitalClient not available - running in CSV-only mode

🧪 TEST MODE: Simulating candles for testing...

🎯 INITIALIZING GOLD TRADING BOT
💰 Account Balance: $10,000.00
📅 Today's Date: 2026-01-26
📅 Yesterday's Date: 2026-01-25
⚠️  No data found for yesterday (2026-01-25) in CSV
   Available dates: ['2026-01-23']
   Using latest available date: 2026-01-23
✅ Loaded GOLD levels from gold_yesterday_levels.csv
   Date: 2026-01-23
⚠️  Error loading CSV: cannot access local variable 'row' where it is not associated with a value

🔄 Could not load from CSV, trying API...
❌ No API client available and CSV not found

❌ Failed to initialize bot


Traceback (most recent call last):
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_21064\2527590461.py", line 64, in load_yesterday_levels
    f"   High: ${y_high:.2f} (avg of bid:${row['prev_high_bid']:.2f}, ask:${row['prev_high_ask']:.2f})"
                                           ^^^
UnboundLocalError: cannot access local variable 'row' where it is not associated with a value


In [ ]:
import ast
import pandas as pd


def add_previous_day_levels(
    df: pd.DataFrame,
    session_start_hour: int | None = None,  # ⬅️ set None to disable shifting
) -> pd.DataFrame:
    """
    Adds previous day high/low (bid & ask) to Capital.com candle data.

    session_start_hour:
        2    → trading day starts at 02:00 (Capital.com default)
        None → use calendar day (00:00)
    """

    df = df.copy()

    # -----------------------------
    # Safe bid / ask extraction
    # -----------------------------
    def parse_price(x):
        if isinstance(x, dict):
            return x
        if isinstance(x, str):
            return ast.literal_eval(x)
        raise ValueError(f"Unexpected price format: {type(x)}")

    for col in ["openPrice", "highPrice", "lowPrice", "closePrice"]:
        parsed = df[col].apply(parse_price)
        df[f"{col}_bid"] = parsed.apply(lambda x: x["bid"])
        df[f"{col}_ask"] = parsed.apply(lambda x: x["ask"])

    # -----------------------------
    # Trading day logic (optional shift)
    # -----------------------------
    ts = pd.to_datetime(df["timestamp"])

    if session_start_hour is not None:
        ts = ts - pd.Timedelta(hours=session_start_hour)

    df["trading_day"] = ts.dt.date

    # -----------------------------
    # Previous day levels
    # -----------------------------
    daily = (
        df.groupby("trading_day")
        .agg(
            prev_high_bid=("highPrice_bid", "max"),
            prev_high_ask=("highPrice_ask", "max"),
            prev_low_bid=("lowPrice_bid", "min"),
            prev_low_ask=("lowPrice_ask", "min"),
        )
        .shift(1)
        .reset_index()
    )

    df = df.merge(daily, on="trading_day", how="left")

    # df.drop(
    #     columns=[
    #         "trading_day",
    #         "openPrice_bid",
    #         "openPrice_ask",
    #         "highPrice_bid",
    #         "highPrice_ask",
    #         "lowPrice_bid",
    #         "lowPrice_ask",
    #         "closePrice_bid",
    #         "closePrice_ask",
    #     ],
    #     inplace=True,
    # )

    return df

In [ ]:
# -------------------------------------------------
# MAIN
# -------------------------------------------------
if __name__ == "__main__":
    client = CapitalClient(
        api_key=os.getenv("CAPITAL_DEMO_API_KEY"),
        identifier=os.getenv("CAPITAL_IDENTIFIER"),
        password=os.getenv("CAPITAL_PASSWORD"),
    )

    client.login()

    df_2025 = client.get_historical_prices(
        epic="GOLD",
        resolution="MINUTE_15",
        from_date="2025-01-01T00:00:00",
        to_date="2025-12-31T23:59:59",
    )

    print(f"\nTotal candles: {len(df_2025)}")

In [ ]:
# df_2025.to_csv("gold_m15_2026_jan_v2.csv", index=False)

In [14]:
updated_df_2025 = add_previous_day_levels(df_2025)

In [18]:
updated_df_2025.to_csv("gold_m15_2026_jan_updated_v2.csv", index=False)

In [20]:
import pandas as pd
from datetime import date
from dataclasses import dataclass


# ====================================================
# 🕯️ BID / ASK CANDLE
# ====================================================
@dataclass
class Candle:
    timestamp: object

    open_bid: float
    open_ask: float

    high_bid: float
    high_ask: float

    low_bid: float
    low_ask: float

    close_bid: float
    close_ask: float


# ====================================================
# 🚀 STRATEGY
# ====================================================
class LiveYesterdayHighLowTrader:
    def __init__(
        self,
        symbol: str,
        risk_per_trade: float,
        tp_pips: float,
        pip_size: float,
        symbol_info: dict,
    ):
        self.symbol = symbol
        self.risk_per_trade = risk_per_trade
        self.tp_pips = tp_pips
        self.pip_size = pip_size
        self.info = symbol_info

        # DAILY STATE
        self.today = None
        self.traded_today = False

        # STRATEGY STATE
        self.y_high = None
        self.y_low = None
        self.break_candle = None
        self.direction = None
        self.prev_candle = None

    # ------------------------------------------------
    # 🔄 DAILY RESET (BID LEVELS)
    # ------------------------------------------------
    def set_new_day(self, today_date: date, yesterday_high_bid, yesterday_low_bid):
        self.today = today_date
        self.y_high = yesterday_high_bid
        self.y_low = yesterday_low_bid

        self.traded_today = False
        self.break_candle = None
        self.direction = None
        self.prev_candle = None

    # ------------------------------------------------
    # 🔥 MAIN ENTRY POINT
    # ------------------------------------------------
    def on_new_candle(self, candle: Candle, capital: float):

        if self.traded_today:
            self.prev_candle = candle
            return None

        if self.prev_candle is None:
            self.prev_candle = candle
            return None

        # ------------------ C1 BREAK (BID) ------------------
        if self.break_candle is None:

            if (
                self.prev_candle.open_bid <= self.y_high
                and self.prev_candle.close_bid > self.y_high
            ):
                self.break_candle = self.prev_candle
                self.direction = "long"

            elif (
                self.prev_candle.open_bid >= self.y_low
                and self.prev_candle.close_bid < self.y_low
            ):
                self.break_candle = self.prev_candle
                self.direction = "short"

            self.prev_candle = candle
            return None

        # ------------------ C2 CONFIRM ------------------
        if self.direction == "long" and candle.close_bid <= self.break_candle.close_bid:
            self._invalidate()
            self.prev_candle = candle
            return None

        if (
            self.direction == "short"
            and candle.close_bid >= self.break_candle.close_bid
        ):
            self._invalidate()
            self.prev_candle = candle
            return None

        # ------------------ C3 ENTRY ------------------
        trade = self._create_trade(candle, capital)
        self.traded_today = True
        self._invalidate()
        self.prev_candle = candle

        return trade

    # ------------------------------------------------
    # 🧮 LOT SIZE
    # ------------------------------------------------
    def _calculate_lot(self, capital, entry, sl):
        risk_amount = capital * self.risk_per_trade
        sl_distance = abs(entry - sl)

        if sl_distance <= 0:
            return 0.0

        ticks = sl_distance / self.info["tick_size"]
        loss_per_lot = ticks * self.info["tick_value"]

        raw_lot = risk_amount / loss_per_lot

        step = self.info["lot_step"]
        min_lot = self.info["min_lot"]

        lot = int(raw_lot / step) * step
        if lot < min_lot:
            lot = min_lot

        return round(lot, 3)

    # ------------------------------------------------
    # 📦 CREATE TRADE (ASK/BID CORRECT)
    # ------------------------------------------------
    def _create_trade(self, candle, capital):

        if self.direction == "long":
            entry = candle.open_ask
            sl = self.break_candle.low_bid
            tp = entry + self.tp_pips * self.pip_size

        else:
            entry = candle.open_bid
            sl = self.break_candle.high_ask
            tp = entry - self.tp_pips * self.pip_size

        lot = self._calculate_lot(capital, entry, sl)

        return {
            "symbol": self.symbol,
            "direction": self.direction,
            "lot": lot,
            "entry_price": entry,
            "stop_loss": sl,
            "take_profit": tp,
            "entry_time": candle.timestamp,
        }

    def _invalidate(self):
        self.break_candle = None
        self.direction = None


# ====================================================
# 🧪 BACKTEST ENGINE
# ====================================================
def backtest_last_year(csv_path, output_csv, initial_capital=10_000):

    df = pd.read_csv(csv_path, parse_dates=["timestamp"])
    df = df.sort_values("timestamp")

    capital = initial_capital
    trades = []

    trader = LiveYesterdayHighLowTrader(
        symbol="XAUUSD",
        risk_per_trade=0.03,
        tp_pips=100,
        pip_size=0.1,
        symbol_info={
            "tick_size": 0.01,
            "tick_value": 1.0,
            "min_lot": 0.001,
            "lot_step": 0.001,
        },
    )

    current_day = None

    for _, row in df.iterrows():
        candle_day = row["timestamp"].date()

        if candle_day != current_day:
            current_day = candle_day
            trader.set_new_day(
                today_date=current_day,
                yesterday_high_bid=row["prev_high_bid"],
                yesterday_low_bid=row["prev_low_bid"],
            )

        candle = Candle(
            timestamp=row["timestamp"],
            open_bid=row["openPrice_bid"],
            open_ask=row["openPrice_ask"],
            high_bid=row["highPrice_bid"],
            high_ask=row["highPrice_ask"],
            low_bid=row["lowPrice_bid"],
            low_ask=row["lowPrice_ask"],
            close_bid=row["closePrice_bid"],
            close_ask=row["closePrice_ask"],
        )

        trade = trader.on_new_candle(candle, capital)

        if trade:
            exit_price, exit_time, result = simulate_exit(trade, df, row["timestamp"])
            pnl = calculate_pnl(trade, exit_price)
            capital += pnl

            trade.update(
                {
                    "exit_price": exit_price,
                    "exit_time": exit_time,
                    "result": result,
                    "pnl": pnl,
                    "equity": capital,
                }
            )
            trades.append(trade)

    pd.DataFrame(trades).to_csv(output_csv, index=False)
    print(f"✅ Backtest finished | Final capital: {round(capital,2)}")


# ====================================================
# 🚪 EXIT SIMULATION (BID/ASK)
# ====================================================
def simulate_exit(trade, df, entry_time):

    future = df[df["timestamp"] > entry_time]

    for _, row in future.iterrows():

        if trade["direction"] == "long":
            if row["lowPrice_bid"] <= trade["stop_loss"]:
                return trade["stop_loss"], row["timestamp"], "loss"
            if row["highPrice_bid"] >= trade["take_profit"]:
                return trade["take_profit"], row["timestamp"], "win"

        else:
            if row["highPrice_ask"] >= trade["stop_loss"]:
                return trade["stop_loss"], row["timestamp"], "loss"
            if row["lowPrice_ask"] <= trade["take_profit"]:
                return trade["take_profit"], row["timestamp"], "win"

    return None, None, "open"


# ====================================================
# 💰 PnL
# ====================================================
def calculate_pnl(trade, exit_price):

    if exit_price is None:
        return 0.0

    move = (
        exit_price - trade["entry_price"]
        if trade["direction"] == "long"
        else trade["entry_price"] - exit_price
    )

    return move / 0.01 * trade["lot"] * 1.0


def generate_report(trades: pd.DataFrame, initial_capital: float):
    df = trades.copy()

    # Remove open trades
    df = df[df["result"].isin(["win", "loss"])]

    if df.empty:
        return {"message": "No closed trades to analyze"}

    wins = df[df["result"] == "win"]
    losses = df[df["result"] == "loss"]

    net_pnl = df["pnl"].sum()
    final_capital = initial_capital + net_pnl

    equity = df["equity"]

    report = {
        "initial_capital": round(initial_capital, 3),
        "final_capital": round(final_capital, 3),
        "net_pnl": round(net_pnl, 3),
        "return_%": round(net_pnl / initial_capital * 100, 3),
        "total_trades": len(df),
        "wins": len(wins),
        "losses": len(losses),
        "win_rate_%": round(len(wins) / len(df) * 100, 3),
        "avg_win": round(wins["pnl"].mean(), 3) if not wins.empty else 0,
        "avg_loss": round(losses["pnl"].mean(), 3) if not losses.empty else 0,
        "expectancy": round(df["pnl"].mean(), 3),
        "profit_factor": (
            round(wins["pnl"].sum() / abs(losses["pnl"].sum()), 3)
            if not losses.empty
            else float("inf")
        ),
        "max_drawdown_%": round(_max_drawdown_pct(equity), 3),
    }

    return report


def _max_drawdown_pct(equity):
    peak = equity.cummax()
    drawdown = (equity - peak) / peak
    return drawdown.min() * 100

In [21]:
# ====================================================
# ▶️ RUN
# ====================================================
if __name__ == "__main__":

    DATA_CSV = "gold_m15_2026_jan_updated_v2.csv"
    TRADE_BOOK_CSV = "trade_book_M15_bid_ask_v2.csv"
    INITIAL_CAPITAL = 100

    backtest_last_year(
        csv_path=DATA_CSV,
        output_csv=TRADE_BOOK_CSV,
        initial_capital=INITIAL_CAPITAL,
    )

    # -----------------------------
    # LOAD TRADE BOOK
    # -----------------------------
    trades_df = pd.read_csv(TRADE_BOOK_CSV, parse_dates=["entry_time", "exit_time"])

    # -----------------------------
    # GENERATE REPORT
    # -----------------------------
    report = generate_report(
        trades=trades_df,
        initial_capital=INITIAL_CAPITAL,
    )

    # -----------------------------
    # PRINT REPORT
    # -----------------------------
    print("\n📊 PERFORMANCE SUMMARY")
    print("-" * 40)
    for k, v in report.items():
        print(f"{k:20}: {v}")

    # -----------------------------
    # SAVE REPORT
    # -----------------------------
    report_df = pd.DataFrame([report])
    report_df.to_csv("summary_M15_2025.csv", index=False)

    print("\n✅ Backtest complete")
    print("📁 Trade book  :", TRADE_BOOK_CSV)
    print("📁 Summary    : summary_2025.csv")

✅ Backtest finished | Final capital: 1264.54

📊 PERFORMANCE SUMMARY
----------------------------------------
initial_capital     : 100
final_capital       : 1264.54
net_pnl             : 1164.54
return_%            : 1164.54
total_trades        : 173
wins                : 107
losses              : 66
win_rate_%          : 61.85
avg_win             : 17.234
avg_loss            : -10.295
expectancy          : 6.731
profit_factor       : 2.714
max_drawdown_%      : -13.551

✅ Backtest complete
📁 Trade book  : trade_book_M15_bid_ask_v2.csv
📁 Summary    : summary_2025.csv


In [7]:
def on_m15_close(candle):
    print(
        f"🕒 {candle['epic']} M15 CLOSED | "
        f"O={candle['o']} H={candle['h']} "
        f"L={candle['l']} C={candle['c']}"
    )
    # run indicators / trades here

In [ ]:
client = CapitalClient(
    api_key=os.getenv("CAPITAL_DEMO_API_KEY"),
    identifier=os.getenv("CAPITAL_IDENTIFIER"),
    password=os.getenv("CAPITAL_PASSWORD"),
)

client.login()

ws = client.stream_ohlc(
    epics=["BTCUSD"],
    resolution="MINUTE_15",
    on_candle_close=on_m15_close,
)

✅ Logged in successfully
▶️ WebSocket thread started


📡 Subscribed to ['GC.D.GOLD.CFD.IP'] MINUTE_15
❌ WS error: Connection to remote host was lost.
🔌 WebSocket closed


In [5]:
client = CapitalClient(
    api_key=os.getenv("CAPITAL_DEMO_API_KEY"),
    identifier=os.getenv("CAPITAL_IDENTIFIER"),
    password=os.getenv("CAPITAL_PASSWORD"),
)

client.login()

r = requests.get(
    f"{client.base_url}/markets",
    headers=client.headers,
    params={"searchTerm": "Gold"},
)
r.raise_for_status()

for m in r.json()["markets"]:
    print(m["epic"], "-", m["instrumentName"])

✅ Logged in successfully
GOLD - Gold
GOLDUS - Gold.com Inc
GCG2026 - Gold - Feb 2026
GS - Goldman Sachs
GOLB - Market Access NYSE Arca Gold Bugs Index UCITS ETF
GCJ2026 - Gold - Apr 2026
GLD - SPDR Gold Shares
NEM - Newmont Goldcorp
RGLD - Royal Gold
GDEN - Golden Entertainment Inc


In [ ]:
while True:
    time.sleep(60)